## Transform annotated json files

In [ ]:
data_annotated = pd.read_json(data_path +'/manually_annotated_dataset.json')

In [1]:
import re
def pad(text):
    text = re.sub("([.,!?(')])", r' \1 ', text)
    text = re.sub('\s{2,}', ' ', text)
    return text

In [ ]:
def get_tokens(text):
    tokens = pad(text).split(' ')
    return tokens

In [ ]:
def nearest_entities(text, e1, e2):
    occ1 = [i for i in range(len(text)) if text.startswith(e1 , i)]
    occ2 = [i for i in range(len(text)) if text.startswith(e2, i)]
    
    minimum = len(text)
    indexes = (0,0)

    for i1,v1 in enumerate(occ1):
        for i2,v2 in enumerate(occ2):
            if abs(v1-v2) < minimum:
                minimum = abs(v1-v2)
                
                indexes = (i1,i2)
    tok_list = get_tokens(text)
    occ1_tokens = []
    for i,x in enumerate(tok_list):
        if len(get_tokens(e1)) == 1:
            if x == get_tokens(e1)[0]:
                occ1_tokens.append(i)
        elif len(get_tokens(e1)) == 2:
            if x == get_tokens(e1)[0] and tok_list[i+1] == get_tokens(e1)[1]:
                occ1_tokens.append(i)
        else:
            if x == get_tokens(e1)[0] and tok_list[i+1] == get_tokens(e1)[1] and tok_list[i+2] == get_tokens(e1)[2]:
                occ1_tokens.append(i)
    occ2_tokens = []
    for i,x in enumerate(tok_list):
        if len(get_tokens(e2)) == 1:
            if x == get_tokens(e2)[0]:
                occ2_tokens.append(i)
        elif len(get_tokens(e2)) == 2:
            if x == get_tokens(e2)[0] and tok_list[i+1] == get_tokens(e2)[1]:
                occ2_tokens.append(i)
        else:
            if x == get_tokens(e2)[0] and tok_list[i+1] == get_tokens(e2)[1] and tok_list[i+2] == get_tokens(e2)[2]:
                occ2_tokens.append(i)
    return occ1[indexes[0]], occ2[indexes[1]], occ1_tokens[indexes[0]], occ2_tokens[indexes[1]]

In [ ]:
def create_dict(df_row):
    tokens_list = get_tokens(df_row['SentText'])
    text = df_row['SentText']
    diz = {"document" : text,
           "tokens": [],
           "relations" : []}
    entities_list = []
    for rel in df_row['RelationMentions']:
        entity_1 = rel['Arg1Text']
        entity_2 = rel['Arg2Text']
        if entity_1 not in text or entity_2 not in text:
            continue
         
        tupla = nearest_entities(text, entity_1, entity_2)
        #print(tupla)


        if entity_1 not in df_row['EntityMentions']:
            continue
        ent1_diz = {"text": entity_1,
                                "start": tupla[0],
                                "end": tupla[0] + len(entity_1),
                                "token_start": tupla[2],
                                "token_end": tupla[2] + len(get_tokens(entity_1)) -1,
                                "entityLabel": rel['RelationNames'][0].split('/')[2].upper()}
        
        

        ent2_diz = {"text": entity_2,
                                "start": tupla[1],
                                "end": tupla[1] + len(entity_2),
                                "token_start": tupla[3],
                                "token_end": tupla[3] + len(get_tokens(entity_2)) -1,
                                "entityLabel": 'IMPUTATO'}
        diz['relations'].append({ "child": ent2_diz["token_start"], "head": ent1_diz["token_start"], "relationLabel": rel['RelationNames'][0].split('/')[1].upper() })
        if ent2_diz not in diz['tokens']:
            diz['tokens'].append(ent2_diz)
        if ent1_diz not in diz['tokens']:

            diz['tokens'].append(ent1_diz)
            
            

        
        
        
    return diz

In [ ]:
for i in range(data_annotated.shape[0]):
    try:
        create_dict(data_annotated.iloc[i])
    except:
        print(i)


0
15


In [ ]:
data_annotated = data_annotated.drop([0,15])
data_annotated

,SentText,EntityMentions,RelationMentions
1,la corte di appello di campobasso confermava ...,"[corte di appello, iuliano nadia]","[{'Arg1Text': 'corte di appello', 'Arg2Text': ..."
2,"11 tribunale di pavia, con sentenza 4 , su ri...","[tribunale, borsan maricel, socio marius]","[{'Arg1Text': 'tribunale', 'Arg2Text': 'borsan..."
3,la corte di appello di firenze ha confermato ...,"[corte di appello, giuseppe patruno]","[{'Arg1Text': 'corte di appello', 'Arg2Text': ..."
4,"il tribunale di milano, in riforma della sent...","[tribunale, herrera jara luis]","[{'Arg1Text': 'tribunale', 'Arg2Text': 'herrer..."
5,"la corte di appello di perugia, in parziale r...","[corte di appello, hudorovich simonetta]","[{'Arg1Text': 'corte di appello', 'Arg2Text': ..."
...,...,...,...
166,con la sentenza impugnata la corte di appello...,"[corte di appello, albert mecja]","[{'Arg1Text': 'corte di appello', 'Arg2Text': ..."
167,"la corte di appello di ancona, con la sentenz...","[tribunale, villani greta]","[{'Arg1Text': 'tribunale', 'Arg2Text': 'villan..."
168,"francesco mastrogiaconno, attraverso il propr...","[francesco mastrogiaconno, difensore]","[{'Arg1Text': 'difensore', 'Arg2Text': 'france..."
169,"con l ordinanza indicata in epigrafe, emessa ...","[tribunale, castagnini renato]","[{'Arg1Text': 'tribunale', 'Arg2Text': 'castag..."


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_annotated, dev_test = train_test_split(data_annotated, test_size=0.3)
dev_annotated, test_annotated = train_test_split(dev_test, test_size=0.5)

In [ ]:
import json
def transform_json(df,filename):
    output = []
    for i in range(df.shape[0]):
        output.append(create_dict(df.iloc[i]))
    with open(data_path + f'/{filename}.txt', 'w') as fout:
        json.dump(output, fout)

In [ ]:
transform_json(data_annotated, 'prova')

In [ ]:
df = pd.read_json(data_path +'/prova.txt')

In [ ]:
df.iloc[44]['document']

' con la sentenza impugnata la corte d appello di venezia ha disposto la consegna all autorità giudiziaria tedesca di ioan rosus, in esecuzione di un mandato di arresto europeo processuale emesso in data 12 agosto 2021 dal penale 6 45207 an021presidente: criscuolo annarelatore: d arcangelo fabriziodata udienza: 06/12/2021tribunale di wuppertal per il reato di furto in abitazione asseritamente commesso dal soggetto richiesto in consegna in data 22/12/2021 a velbert. 2. l '

In [ ]:
distances = []
for i in range(df.shape[0]):
    for diz in df.iloc[i]['relations']:
        m = abs(diz['head'] - diz['child'])
        distances.append(m)
print(distances)
print(max(distances))

[21, 17, 20, 10, 26, 68, 7, 7, 11, 14, 14, 15, 10, 17, 21, 10, 22, 15, 23, 22, 6, 13, 18, 22, 26, 7, 6, 11, 21, 18, 17, 28, 17, 20, 23, 20, 42, 19, 22, 9, 20, 28, 13, 17, 2, 25, 6, 23, 8, 11, 14, 14, 25, 25, 15, 40, 19, 2, 23, 27, 2, 27, 17, 22, 6, 6, 39, 20, 6, 18, 23, 9, 19, 8, 7, 23, 25, 18, 70, 60, 50, 9, 7, 14, 9, 2, 8, 11, 7, 16, 14, 2, 7, 28, 16, 16, 15, 16, 5, 5, 8, 2, 19, 23, 33, 32, 30, 12, 6, 6, 32, 16, 12, 8, 20, 22, 18, 19, 22, 15, 11, 9, 30, 33, 28, 31, 34, 37, 4, 12, 31, 18, 11, 5, 27, 27, 11, 9, 9, 39, 32, 9, 12, 23, 11, 14, 17, 20, 23, 26, 22, 11, 25, 14, 18, 22, 7, 12, 60, 50, 67, 33, 36, 39, 42, 21, 22, 35, 19, 2, 4, 35, 39, 43, 9, 12, 22, 9, 6, 6, 6, 19, 24, 7, 20, 19, 31, 18, 18, 13, 25, 13, 6, 25, 8, 26, 6, 6, 12, 9]
70


In [ ]:
transform_json(train_annotated, 'relation_training')
transform_json(test_annotated, 'relation_test')
transform_json(dev_annotated, 'relation_dev')

## Convert json files to spacy binary files

In [ ]:
import json


from pathlib import Path

from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
from spacy.tokenizer import Tokenizer
#from spacy.lang.en import English
from spacy.util import compile_infix_regex
import re
import spacy

nlp = spacy.blank("it")
# Create a blank Tokenizer with just the English vocab

msg = Printer()

SYMM_LABELS = ["Binds"]
MAP_LABELS = {
    "DIFENDE": "DIFENDE",
    "GIUDICA": "GIUDICA"
}


def main(json_loc: Path, output_file: Path):
    """Creating the corpus from the Prodigy annotations."""
    Doc.set_extension("rel", default={},force=True)
    vocab = Vocab()

    docs = {"train": [], "dev": [], "test": [], "total": []}
    ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
    count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
    count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}

    with open(json_loc, encoding="utf8") as jsonfile:
        file = json.load(jsonfile)
        i = 0
        for example in file:
            span_starts = set()
            neg = 0
            pos = 0
                    # Parse the tokens
            tokens=nlp(example["document"])    

            spaces=[]
            spaces = [True if tok.whitespace_ else False for tok in tokens]
            words = [t.text for t in tokens]
            doc = Doc(nlp.vocab, words=words, spaces=spaces)


            # Parse the GGP entities
            spans = example["tokens"]
            entities = []
            span_end_to_start = {}
            for span in spans:
                entity = doc.char_span(
                     span["start"], span["end"], label=span["entityLabel"]
                 )
                

                span_end_to_start[span["token_start"]] = span["token_start"]
                #print(span_end_to_start)
                entities.append(entity)
                span_starts.add(span["token_start"])
            #print(entities)
            doc.ents = entities
            #print(i)
            i += 1

            # Parse the relations
            rels = {}
            for x1 in span_starts:
                for x2 in span_starts:
                    rels[(x1, x2)] = {}
                    #print(rels)
            relations = example["relations"]
            #print(len(relations))
            for relation in relations:
                # the 'head' and 'child' annotations refer to the end token in the span
                # but we want the first token
                start = span_end_to_start[relation["head"]]
                end = span_end_to_start[relation["child"]]
                label = relation["relationLabel"]
                #print(rels[(start, end)])
                #print(label)
                #label = MAP_LABELS[label]
                if label not in rels[(start, end)]:
                    rels[(start, end)][label] = 1.0
                    pos += 1
                    #print(pos)
                    #print(rels[(start, end)])

            # The annotation is complete, so fill in zero's where the data is missing
            for x1 in span_starts:
                for x2 in span_starts:
                    for label in MAP_LABELS.values():
                        if label not in rels[(x1, x2)]:
                            neg += 1
                            rels[(x1, x2)][label] = 0.0

                            #print(rels[(x1, x2)])
            doc._.rel = rels
            #print(doc._.rel)

            # only keeping documents with at least 1 positive case
            if pos > 0:
                    docs["total"].append(doc)
                    count_pos["total"] += pos
                    count_all["total"] += pos + neg

                    
                    
    #print(len(docs["total"]))
    docbin = DocBin(docs=docs["total"], store_user_data=True)
    docbin.to_disk(output_file)
    msg.info(
        f"{len(docs['total'])} training sentences"
    )
    #print(docs)

In [ ]:
train_file = data_path + '/train.spacy'
dev_file = data_path + '/dev.spacy'
test_file = data_path + '/test.spacy'

In [ ]:
main(data_path +'/relation_test.txt', test_file)

ℹ 26 training sentences


In [ ]:
main(data_path +'/relation_dev.txt', dev_file)

ℹ 25 training sentences


In [ ]:
main(data_path +'/relation_training.txt', train_file)

ℹ 118 training sentences
